In [8]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import os
import glob 

In [4]:
start_time = datetime.now()

In [5]:
dir = 'Y:\\delete\\Australian Government Department of Health'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [11]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\Australian Government Department of Health"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://www.industrialchemicals.gov.au/search-inventory")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value='//*[@id="block-nicnas-theme-mainpagecontent"]/div/article/div/div[4]/div/div[1]/div[1]/div/div/div[1]/article/div/a').click()
time.sleep(10)
driver.close()

In [5]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:00:08.213898


In [45]:
path = 'Y:\\delete\\Australian Government Department of Health'
filenames = glob.glob(path + "/*.xlsx")
for f in filenames:
    dataset = pd.read_excel(f,skiprows=1)
dataset = dataset[['CAS No','Specific information requirement\n']]
dataset.columns.values[0] = "CAS Number"
dataset.columns.values[1] = "status"
dataset = dataset.loc[dataset['CAS Number'] != 'Confidential']
dataset["status"] = dataset["status"].notnull().astype("int")
dataset['status'] = dataset['status'].astype("string")
dataset = dataset[dataset['CAS Number'].notna()]
dataset.head(2)

,CAS Number,status
0,6093-03-4,0
2,93507-51-8,0


In [46]:
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Not in list'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.status.fillna(result.value, inplace=True)
del result['value']

,CAS Number,status
0,6093-03-4,0
1,93507-51-8,0
2,92113-15-0,0
3,10016-20-3,0
4,128446-33-3,0
...,...,...
38998,101033-44-7,0
38999,23940-69-4,0
39000,102958-43-0,0
39001,57534-41-5,0


In [48]:
result['Jurisdiction'] = 'Australia'
result['RegulatoryAgency'] = 'Department of Health'
result['TypeOfRegulation'] = 'Use'
result['Criteria'] = 'Different use circumstances versus assessment report'

result['Description'] = 'Not on the inventory'
result.loc[result['status'] == '0', 'Description'] = 'No specific information requrement applies'
result.loc[result['status'] == '1', 'Description'] = 'Specific information requrement applies'

result['HazardClassification'] = 'Low'
result.loc[result['status'] == '0', 'HazardClassification'] = 'Moderate'
result.loc[result['status'] == '1', 'HazardClassification'] = 'High'

result['CauseOfClassification'] = 'Not on the inventory'
result.loc[result['status'] == '0', 'CauseOfClassification'] = 'No notification of manufacture/import is required'
result.loc[result['status'] == '1', 'CauseOfClassification'] = 'Notification of different use circumstnaces is required'

result['SourceName'] = 'Australian Government Department of Health'
result['SourceURL'] = 'https://www.industrialchemicals.gov.au/search-inventory'

result.head(2)

,CAS Number,status,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,6093-03-4,0,Australia,Department of Health,Use,Different use circumstances versus assessment ...,No specific information requrement applies,Moderate,No notification of manufacture/import is required,Australian Government Department of Health,https://www.industrialchemicals.gov.au/search-...
1,93507-51-8,0,Australia,Department of Health,Use,Different use circumstances versus assessment ...,No specific information requrement applies,Moderate,No notification of manufacture/import is required,Australian Government Department of Health,https://www.industrialchemicals.gov.au/search-...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [ ]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))